# Day 2 課題

### 目次
### - 1. Day 1 の作業内容
### - 2. Day 1で構築した予測モデルの汎化性能検証
### - 3. 前処理について
### - 4. サポートベクタ―マシンによる予測モデル構築

## 1. Day 1 の作業内容

### 各変数について
- ID：不要と判断し削除
- name：用いる変数の中ではnameにのみ欠損値があった。欠損値が少ないため、削除して対応した。
- name_length：nameの単語数と成功確率に相関があるのではないかと考え、nameの単語数の変数を追加した。
- category：カテゴリーの種類によって成功確率に違いが見られた。ダミー変数を追加し予測に用いた。データ数の少ないカテゴリーもあり、予測に用いた場合、過学習の懸念がある。
- main_category：成功確率との関係が見られた。ダミー変数を追加し、予測に用いた。
- currency：成功確率との関係が見られた。ダミー変数を追加し、予測に用いた。
- country：成功確率との関係が見られた。ダミー変数を追加し、予測に用いたが、データ数の少ないカテゴリーもあり、予測に用いた場合、過学習の懸念がある。
- goal, usd_goal_real：分布について確認すると、goalの最大値100000000.0をとっているデータが多く見られた。これらは正常な値である可能性もあるが、設定された上限の値を超えていて、便宜的に100000000.0となっている可能性がある。そのためgoalがこれらの値となっているデータについては除外することとする。goalよりも通貨が揃っているusd_goal_realの方が適切と判断し、こちらを予測に用いている。
- deadline, launched：開始から、終了までの日数を求め、変数deltaとして追加した。deltaの外れ値について確認し、最大値91日のものがいくつか見られたが、クラウドファンディングの設定可能日数が91日であり、そのまま使用して問題ないと判断した。日数が0のデータも存在したが、実際に開始日と終了日を確認したところ、期間が一日未満であったため、こちらも問題ないと判断した。
- pledged, usd_peldged, usd_pledged_real, backers：クラウドファンディングの結果得られる値のため、今回は使用不可として削除
- state：目的変数、今回は結果の確定しているsuccessful, failedのみを用い、その他の値となっているデータは扱わないこととした。successful=1, failed=0 として数値に置き換えた。 

量的変数に関しては、標準化して計算に用いた。各変数名に_stdを付けたものを新たにdata frameに追加している。

### 各変数の相関
heatmup, 散布図行列を用いて、各変数の相関を調べた。
- name_lengthとstateには正の相関がみられ、名前の情報量が多い方が、成功しやすい傾向が示唆された。
- usd_goal_realとstateの相関係数は小さかった。目標金額と成功確率は関係しているように思われるが、相関係数には現れていない可能性もある。
- usd_goal_realとdeltaには小さな負の相関が見られた。直感に反するが、関係が相関係数で表現できていない可能性もある。

### ロジスティック回帰による予測
以下の二通の説明変数を用いて、stateの値を予測し、訓練誤差を評価した。
1. name_length_std, usd_goal_std, currencyおよび main_categoryのダミー変数, delta_std 
2. 上記にcountry, categoriのダミー変数を追加したもの
1より2の方が予測精度は高かったが、訓練データに過学習している可能性もあるので、汎化性能を評価したい。

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import log_loss, accuracy_score, confusion_matrix
from sklearn.preprocessing import StandardScaler

In [4]:
#各質的変数のダミー変数を追加する前のdf
df_day2_1=pd.read_csv('Day1.csv')
df_day2_1.head()

,name,category,main_category,currency,deadline,goal,launched,state,country,usd_goal_real,usd_goal_real_std,name_length,name_length_std,delta,delta_std
0,Monarch Espresso Bar,Restaurants,Food,USD,2016-04-01 00:00:00,50000.0,2016-02-26 13:38:27,1,US,50000.00,0.025405,3,-0.954203,34,0.082298
1,Support Solar Roasted Coffee & Green Energy! ...,Food,Food,USD,2014-12-21 00:00:00,1000.0,2014-12-01 18:30:44,1,US,1000.00,-0.050121,8,0.859112,19,-1.097677
2,Lisa Lim New CD!,Indie Rock,Music,USD,2013-04-08 00:00:00,12500.0,2013-03-09 06:42:58,1,US,12500.00,-0.032395,4,-0.591540,29,-0.311027
3,Tombstone: Old West tabletop game and miniatur...,Tabletop Games,Games,GBP,2017-05-03 00:00:00,5000.0,2017-04-05 19:44:18,1,GB,6469.73,-0.041690,9,1.221775,27,-0.468357
4,Mike Corey's Darkness & Light Album,Music,Music,USD,2012-08-17 00:00:00,250.0,2012-08-02 14:11:32,1,US,250.00,-0.051277,6,0.133786,14,-1.491002


In [5]:
#currency, main_ategoryのダミー変数を追加したdf
df_day2_2=pd.read_csv('Day1_2.csv')
df_day2_2.head()

,name,category,deadline,goal,launched,state,country,usd_goal_real,usd_goal_real_std,name_length,...,currency_EUR,currency_GBP,currency_HKD,currency_JPY,currency_MXN,currency_NOK,currency_NZD,currency_SEK,currency_SGD,currency_USD
0,Monarch Espresso Bar,Restaurants,2016-04-01 00:00:00,50000.0,2016-02-26 13:38:27,1,US,50000.00,0.025405,3,...,0,0,0,0,0,0,0,0,0,1
1,Support Solar Roasted Coffee & Green Energy! ...,Food,2014-12-21 00:00:00,1000.0,2014-12-01 18:30:44,1,US,1000.00,-0.050121,8,...,0,0,0,0,0,0,0,0,0,1
2,Lisa Lim New CD!,Indie Rock,2013-04-08 00:00:00,12500.0,2013-03-09 06:42:58,1,US,12500.00,-0.032395,4,...,0,0,0,0,0,0,0,0,0,1
3,Tombstone: Old West tabletop game and miniatur...,Tabletop Games,2017-05-03 00:00:00,5000.0,2017-04-05 19:44:18,1,GB,6469.73,-0.041690,9,...,0,1,0,0,0,0,0,0,0,0
4,Mike Corey's Darkness & Light Album,Music,2012-08-17 00:00:00,250.0,2012-08-02 14:11:32,1,US,250.00,-0.051277,6,...,0,0,0,0,0,0,0,0,0,1


## 2. Day 1で構築した予測モデルの汎化性能検証
#### 交差検証を用いて汎化性能を評価する

In [6]:
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import log_loss, accuracy_score, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix 
from sklearn.model_selection import KFold 

### 2-1 name_length_std, usd_goal_std, delta_std, currencyおよび main_categoryのダミー変数を説明変数とした予測モデルの評価

In [7]:
#必要ない変数を除く
X = df_day2_2.drop(["state", "launched", "country", "usd_goal_real", "name", "name_length", "category", "deadline", "goal","delta"], axis=1)
y = df_day2_2["state"]
X.head()

,usd_goal_real_std,name_length_std,delta_std,main_category_Art,main_category_Comics,main_category_Crafts,main_category_Dance,main_category_Design,main_category_Fashion,main_category_Film & Video,...,currency_EUR,currency_GBP,currency_HKD,currency_JPY,currency_MXN,currency_NOK,currency_NZD,currency_SEK,currency_SGD,currency_USD
0,0.025405,-0.954203,0.082298,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,-0.050121,0.859112,-1.097677,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,-0.032395,-0.591540,-0.311027,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,-0.041690,1.221775,-0.468357,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,-0.051277,0.133786,-1.491002,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [8]:
# 交差検証法により、汎化性能を検証する。
n_split = 5 #データの分割数
result=[]#各回の正答率を格納する配列
split_num = 1

for train_idx, test_idx in KFold(n_splits=n_split, shuffle=True, random_state=1234).split(X, y):
    X_train, y_train = X.iloc[train_idx, :], y[train_idx] #学習用データ
    X_test, y_test = X.iloc[test_idx, :], y[test_idx]     #テスト用データ
    
    #学習用データを用いてモデルの学習を行う。
    clf = SGDClassifier(loss='log', penalty='none', max_iter=10000, fit_intercept=True, random_state=1234, tol=1e-3, )
    clf.fit(X_train, y_train)

    # 訓練データのラベルを予測
    y_est = clf.predict(X_train)
    # 確率値を得る
    y_est_proba = clf.predict_proba(X_train)
    # 対数尤度を計算
    llr = - log_loss(y_train, y_est_proba, normalize=False)
    # 正答率を計算
    accuracy=100 * accuracy_score(y_train, y_est)

    # Precision, Recall, F1-scoreを計算
    precision, recall, f1_score, _ = precision_recall_fscore_support(y_train, y_est)

    result.append(["Fold %s 訓練"%split_num, llr, accuracy, 100 * precision[1], 100 * recall[1], 100 * f1_score[1]])

    # テストデータのラベルを予測
    y_est = clf.predict(X_test)
    # 確率値を得る
    y_est_proba = clf.predict_proba(X_test)    
    # 対数尤度を計算
    llr = - log_loss(y_test, y_est_proba, normalize=False)
    # 正答率を計算
    accuracy=100 * accuracy_score(y_test, y_est)
    # Precision, Recall, F1-scoreを計算
    precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, y_est)
   
    result.append(["Fold %s テスト"%split_num, llr, accuracy, 100 * precision[1], 100 * recall[1], 100 * f1_score[1]])
    split_num += 1

#各回のデータが入った配列をdataframeにし、訓練データ、テストデータそれぞれで各値の平均を求める。
result_df = pd.DataFrame(result,columns=['name', '対数尤度', '正答率', '適合率', '再現率', 'F1値'],)
result_df.set_index('name')
display(result_df)
test_result = result_df.iloc[[1, 3, 5, 7, 9], : ]
train_result = result_df.iloc[[0, 2, 4, 6, 8], : ]

print("訓練データ Cross Validation")
display(train_result.mean())

print("テストデータ Cross Validation")
display(test_result.mean())


,name,対数尤度,正答率,適合率,再現率,F1値
0,Fold 1 訓練,-164978.725565,64.993065,60.695243,38.183580,46.876805
1,Fold 1 テスト,-41261.642345,65.017338,60.257100,37.837432,46.485240
2,Fold 2 訓練,-164973.455694,64.926107,60.036525,39.279913,47.489208
3,Fold 2 テスト,-41348.509196,64.731867,59.660464,39.556467,47.571662
4,Fold 3 訓練,-165070.541192,64.934776,60.826085,36.664829,45.751504
5,Fold 3 テスト,-41257.262583,64.808756,61.151212,36.769670,45.925034
6,Fold 4 訓練,-164964.736963,64.941184,59.738075,40.148188,48.022128
7,Fold 4 テスト,-41239.387393,64.826848,60.004446,40.089120,48.065536
8,Fold 5 訓練,-165426.671059,64.684133,60.913844,35.491654,44.850825
9,Fold 5 テスト,-41262.162206,64.947459,60.843140,35.370621,44.734966


訓練データ Cross Validation


対数尤度   -165082.826095
正答率         64.895853
適合率         60.441954
再現率         37.953633
F1値         46.598094
dtype: float64

テストデータ Cross Validation


対数尤度   -41273.792745
正答率        64.866454
適合率        60.383273
再現率        37.924662
F1値        46.556487
dtype: float64

訓練データにおける精度、テストデータにおける誤差がほぼ変わらず、過学習は起こしていないように思われる。対数尤度の合計値も訓練データがテストデータの4倍の量であるから、合計値がほぼ4倍になっており、差はほとんど見られない。

### 2-2 country, categoryのダミー変数を説明変数に追加した予測モデルの評価

In [9]:
#ダミー変数の追加
df_day2_3=pd.get_dummies(df_day2_2, columns=["country"])
df_day2_3=pd.get_dummies(df_day2_3, columns=["category"])

In [10]:
#必要ない変数を除く
X2 = df_day2_3.drop(["state", "launched", "usd_goal_real", "name", "name_length", "deadline", "goal","delta"], axis=1)
y2 = df_day2_3["state"]
X2.head()

,usd_goal_real_std,name_length_std,delta_std,main_category_Art,main_category_Comics,main_category_Crafts,main_category_Dance,main_category_Design,main_category_Fashion,main_category_Film & Video,...,category_Wearables,category_Weaving,category_Web,category_Webcomics,category_Webseries,category_Woodworking,category_Workshops,category_World Music,category_Young Adult,category_Zines
0,0.025405,-0.954203,0.082298,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,-0.050121,0.859112,-1.097677,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,-0.032395,-0.591540,-0.311027,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,-0.041690,1.221775,-0.468357,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,-0.051277,0.133786,-1.491002,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
# 交差検証法により、汎化性能を検証する。
n_split = 5 
result=[]
split_num = 1

for train_idx, test_idx in KFold(n_splits=n_split, shuffle=True, random_state=1234).split(X, y):
    X_train, y_train = X2.iloc[train_idx, :], y2[train_idx] 
    X_test, y_test = X2.iloc[test_idx, :], y2[test_idx]    
    
    clf = SGDClassifier(loss='log', penalty='none', max_iter=10000, fit_intercept=True, random_state=1234, tol=1e-3, )
    clf.fit(X_train, y_train)
    
    y_est = clf.predict(X_train)
    y_est_proba = clf.predict_proba(X_train)
    llr = - log_loss(y_train, y_est_proba, normalize=False)
    accuracy=100 * accuracy_score(y_train, y_est)
    precision, recall, f1_score, _ = precision_recall_fscore_support(y_train, y_est)
    result.append(["Fold %s 訓練"%split_num, llr, accuracy, 100 * precision[1], 100 * recall[1], 100 * f1_score[1]])

    y_est = clf.predict(X_test)
    y_est_proba = clf.predict_proba(X_test)    
    llr = - log_loss(y_test, y_est_proba, normalize=False)
    accuracy=100 * accuracy_score(y_test, y_est)
    precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, y_est)
    result.append(["Fold %s テスト"%split_num, llr, accuracy, 100 * precision[1], 100 * recall[1], 100 * f1_score[1]])

    split_num += 1

result_df = pd.DataFrame(result,columns=['name', '対数尤度', '正答率', '適合率', '再現率', 'F1値'],)
result_df.set_index('name')
display(result_df)
test_result = result_df.iloc[[1, 3, 5, 7, 9], : ]
train_result = result_df.iloc[[0, 2, 4, 6, 8], : ]

print("訓練データ Cross Validation")
display(train_result.mean())

print("テストデータ Cross Validation")
display(test_result.mean())

,name,対数尤度,正答率,適合率,再現率,F1値
0,Fold 1 訓練,-159121.583347,67.122601,63.482030,44.076183,52.028487
1,Fold 1 テスト,-39861.756284,67.194331,63.111183,44.051061,51.886083
2,Fold 2 訓練,-158994.509272,67.226751,62.389908,47.412393,53.879651
3,Fold 2 テスト,-39939.356734,66.978245,62.007116,47.417070,53.739413
4,Fold 3 訓練,-159299.562509,67.160039,61.692737,48.987374,54.610808
5,Fold 3 テスト,-39766.048180,67.038550,62.059659,48.621879,54.525033
6,Fold 4 訓練,-159171.385357,67.184538,62.601164,46.327061,53.248420
7,Fold 4 テスト,-39745.543708,67.133531,62.933643,46.346082,53.380950
8,Fold 5 訓練,-159345.047609,67.071465,62.708394,45.934868,53.026792
9,Fold 5 テスト,-39848.528021,67.300879,62.595459,45.906631,52.967581


訓練データ Cross Validation


対数尤度   -159186.417619
正答率         67.153079
適合率         62.574847
再現率         46.547576
F1値         53.358832
dtype: float64

テストデータ Cross Validation


対数尤度   -39832.246585
正答率        67.129107
適合率        62.541412
再現率        46.468545
F1値        53.299812
dtype: float64

テストデータにおける精度がわずかに低くなっているが、明らかに過学習が起きているとは言えない。説明変数の追加によって精度が数%上昇している。

## 3. ロジスティック回帰の正則化項の検討

### 2-2 でより高い精度を実現した説明変数を採用する。
#### ここではホールドアウト法で検証する。

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
#データの分割　正則化項の強さのチューニングを行うため、最終テスト用のテストデータを残している。
X_train, X_final_test, y_train, y_final_test = train_test_split(X2, y2, test_size = 0.1, random_state=1234)
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size = 0.1, random_state=1234)

In [10]:
#予測結果を出力する関数
def print_result(y, y_est, y_est_proba):
    precision, recall, f1_score, _ = precision_recall_fscore_support(y, y_est)
    print('対数尤度 = {:.3f}'.format(- log_loss(y, y_est_proba, normalize=False))) 
    print('正答率 = {:.3f}%'.format(100 * accuracy_score(y, y_est)))
    print('適合率（Precision） = {:.3f}%'.format(100 * precision[1]))
    print('再現率（Recall） = {:.3f}%'.format(100 * recall[1]))
    print('F1値（F1-score） = {:.3f}%'.format(100 * f1_score[1]))
    
    return 

In [11]:
#検証用の関数を定義する。データ、正則化項（l1 or l2),正則化の強さalphaを引数としている。
#戻り値は重みのリスト、対数尤度、正答率、訓練済みモデルである。
def reguralization(X_train, X_test, y_train, y_test, penalty, alpha):
    clf = SGDClassifier(loss='log', penalty=penalty, alpha=alpha,  max_iter=10000, fit_intercept=True, random_state=1234, tol=1e-3, )
    clf.fit(X_train, y_train)

    # 重みを取得
    w0 = clf.intercept_[0]
    w = clf.coef_

    # 訓練データのラベルを予測
    y_est = clf.predict(X_train)
    y_est_proba = clf.predict_proba(X_train)
    print("訓練データ")
    print_result(y_train, y_est, y_est_proba)
    
    # テストデータのラベルを予測
    y_est = clf.predict(X_test)
    y_est_proba = clf.predict_proba(X_test)
    print("\nテストデータ")
    print_result(y_test, y_est, y_est_proba)

    return w, - log_loss(y_test, y_est_proba, normalize=False), 100 * accuracy_score(y_test, y_est), clf

In [12]:
#パラメータチューニング用の関数。上記の関数を簡略化したもの。
def reguralization2(X_train, X_test, y_train, y_test, penalty, alpha):
    clf = SGDClassifier(loss='log', penalty=penalty, alpha=alpha,  max_iter=10000, fit_intercept=True, random_state=1234, tol=1e-3, )
    clf.fit(X_train, y_train)

    # 重みを取得
    w0 = clf.intercept_[0]
    w = clf.coef_

    # テストデータのラベルを予測
    y_est = clf.predict(X_test)
    y_est_proba = clf.predict_proba(X_test)

    return  w, - log_loss(y_test, y_est_proba, normalize=False), 100 * accuracy_score(y_test, y_est), clf

比較のため、正則化なしのモデルも同じ訓練データで学習し、最終テストデータでの予測精度を評価する。
重みについても確認する。

In [13]:
w_0, ll_0, a_0, clf_0 = reguralization(X_train, X_final_test, y_train, y_final_test, "none", 1)
w_0

訓練データ
対数尤度 = -173921.190
正答率 = 62.024%
適合率（Precision） = 56.651%
再現率（Recall） = 25.812%
F1値（F1-score） = 35.465%

テストデータ
対数尤度 = -21428.033
正答率 = 62.090%
適合率（Precision） = 55.630%
再現率（Recall） = 25.775%
F1値（F1-score） = 35.227%


array([[-1.29248899e-01,  2.47286952e-01, -2.41946889e-01,
         1.03723225e-01,  8.28840497e-02, -3.40880310e-02,
         2.69418933e-03, -1.05066457e-01, -8.20010047e-02,
        -3.49070352e-02, -5.37569435e-01,  2.02055729e-01,
        -3.97034442e-02,  1.32041282e-01, -2.97959130e-02,
        -3.81774913e-02, -1.90905988e-01,  3.09730155e-01,
        -2.37659720e-02, -6.22403462e-02, -5.41508253e-03,
        -4.99589228e-04, -7.20632124e-02, -2.54726243e-02,
        -1.26085868e-04, -6.62276200e-05, -4.78226742e-03,
        -4.45780913e-03, -1.11079716e-02, -1.11157020e-03,
         9.54666514e-05, -4.80728785e-02, -1.62305445e-03,
        -2.37523778e-02, -3.23947228e-03, -6.22468924e-02,
        -5.41508253e-03, -1.45295927e-02, -5.13866879e-04,
        -1.44680669e-02, -1.20698627e-02, -2.55225965e-02,
        -1.26085868e-04, -1.57106747e-02, -8.85268948e-03,
        -6.62276200e-05, -1.34879061e-05, -4.78226742e-03,
         1.32747953e-04, -1.40581992e-03, -4.43090904e-0

#### まずはL1正則化を試す。

In [14]:
#alpha = 1
w1,ll1,a1,clf1 = reguralization(X_train, X_test, y_train, y_test, "l1", 1)
w1

訓練データ
対数尤度 = -181247.847
正答率 = 59.573%
適合率（Precision） = 0.000%
再現率（Recall） = 0.000%
F1値（F1-score） = 0.000%


/opt/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



テストデータ
対数尤度 = -20148.841
正答率 = 59.485%
適合率（Precision） = 0.000%
再現率（Recall） = 0.000%
F1値（F1-score） = 0.000%


array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.]])

すべてのパラメータの重みがゼロになってしまい、すべて失敗と予測してしまっている。
#### 正則化のパラメータalphaの大きさについて検討してみる。

In [15]:
#alpha = 0.01
w2, ll2, a2, clf2 = reguralization(X_train, X_test, y_train, y_test, "l1", 0.01)

訓練データ
対数尤度 = -176451.376
正答率 = 60.503%
適合率（Precision） = 55.807%
再現率（Recall） = 11.044%
F1値（F1-score） = 18.440%

テストデータ
対数尤度 = -19588.006
正答率 = 60.580%
適合率（Precision） = 56.838%
再現率（Recall） = 11.238%
F1値（F1-score） = 18.766%


In [16]:
#alpha = 0.001
w3, ll3, a3, clf3 = reguralization(X_train, X_test, y_train, y_test, "l1", 0.001)

/opt/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/base.py:340: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)


訓練データ
対数尤度 = -164488.132
正答率 = 66.025%
適合率（Precision） = 61.841%
再現率（Recall） = 41.675%
F1値（F1-score） = 49.794%

テストデータ
対数尤度 = -18219.102
正答率 = 66.199%
適合率（Precision） = 62.398%
再現率（Recall） = 41.702%
F1値（F1-score） = 49.993%


alpha = 0.01, 0.001では学習が上手く行えた。
#### それぞれのalpha値の場合において、予測に用いられた説明変数を確認する。

In [17]:
#説明変数名のリスト, モデルの重みを値に持つdata frameを作成する。
val_df = pd.DataFrame(list(X2.columns),columns=['name'])
val_df['w']=w2.reshape(-1,1)
#重みが大きい順に並べる
val_df = val_df.sort_values('w', ascending=False)
display(val_df.head())
#重みが小さい順に並べる
val_df = val_df.sort_values('w')
display(val_df.head())

,name,w
13,main_category_Music,0.218091
1,name_length_std,0.180437
158,category_Photobooks,0.000000
146,category_Music Videos,0.000000
136,category_Letterpress,0.000000


,name,w
2,delta_std,-0.206539
16,main_category_Technology,-0.148267
0,usd_goal_real_std,-0.019216
105,category_Farmer's Markets,0.000000
14,main_category_Photography,0.000000


In [18]:
val_df2 = pd.DataFrame(list(X2.columns),columns=['name'])
val_df2['w']=w3.reshape(-1,1)
val_df2 = val_df2.sort_values('w', ascending=False)
display(val_df2.head(20))

val_df2 = val_df.sort_values('w')
display(val_df2.head())

,name,w
191,category_Tabletop Games,1.126745
17,main_category_Theater,0.957827
4,main_category_Comics,0.814126
6,main_category_Dance,0.772273
184,category_Shorts,0.662651
13,main_category_Music,0.641764
127,category_Indie Rock,0.320876
3,main_category_Art,0.295062
7,main_category_Design,0.247147
1,name_length_std,0.237619


,name,w
2,delta_std,-0.206539
16,main_category_Technology,-0.148267
0,usd_goal_real_std,-0.019216
136,category_Letterpress,0.000000
137,category_Literary Journals,0.000000


L1正則化の特性通り、予測に用いる説明変数の数が少なくなっていた。少ない説明変数で予測を行う際に参考としたい。
#### 次にパラメータalphaをチューニングする。X_train, y_train, X_test, y_testを用いて最も汎化誤差の少ないパラメータを選び、そのパラメータで最終テストデータを評価し、正則化項なし、L2正則化と比較する。
練習・および計算負荷低下のため、ホールドアウト法で検討しています。交差検証によるパラメータチューニングはSVMの部分で行っています。

[質問]このように交差検証法でなくホールドアウト法を持ちいてパラメータチューニングを行うことはあるのでしょうか？

In [19]:
#alphaを小さくしながら、モデルの構築を繰り返し、それまでの結果より精度が良かった場合、値とモデルを記憶する。
a = 1
ma = 0
alpha_max = 0
w_max = []
ll_max = -float("inf")
clf_best = 0

for i in range(8):
    w_tmp, ll_tmp, a_tmp, clf_tmp = reguralization2(X_train, X_test, y_train, 
                                   y_test, "l1", a)
    if ll_tmp > ll_max:
        ma = a_tmp
        alpha_max = a
        w_max = w_tmp
        ll_max = ll_tmp
        clf_best = clf_tmp
    a /= 10
    
print('テストデータの誤差が最小となるαは：', alpha_max)    
print('その時の正答率 = {:.3f}%'.format(ma))
print('その時の対数尤度 = {:.3f}'.format(ll_max))

/opt/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/base.py:340: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)


テストデータの誤差が最小となるαは： 0.0001
その時の正答率 = 66.963%
その時の対数尤度 = -17902.682


#### alpha=0.0001が最も汎化誤差が小さくなるパラメータであった。これを用いて最終テストを評価する。

In [21]:
# 最終テストデータのラベルを予測
y_est = clf_best.predict(X_final_test)
y_est_proba = clf_best.predict_proba(X_final_test)
print_result(y_final_test, y_est, y_est_proba)

対数尤度 = -19914.193
正答率 = 67.083%
適合率（Precision） = 61.916%
再現率（Recall） = 45.986%
F1値（F1-score） = 52.775%


正則化項なしの下記のデータと比較すると、精度が向上していることがわかる。
- 対数尤度 = -21428.033
- 正答率 = 62.090%
- 適合率（Precision） = 55.630%
- 再現率（Recall） = 25.775%
- F1値（F1-score） = 35.227%

#### 次にL2正則化を試す

In [22]:
w3, a3, ll4, clf4 = reguralization(X_train, X_test, y_train, y_test, "l2", 1)

訓練データ
対数尤度 = -179627.089
正答率 = 59.574%
適合率（Precision） = 100.000%
再現率（Recall） = 0.001%
F1値（F1-score） = 0.002%

テストデータ
対数尤度 = -19961.964
正答率 = 59.485%
適合率（Precision） = 0.000%
再現率（Recall） = 0.000%
F1値（F1-score） = 0.000%


/opt/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


L2正則化でもalphaが1だとほとんどのデータを失敗と予測していると推測できる。
#### テストデータの正答率を最大にするalphaの値を求めてみる。

In [23]:
a = 1
ma = 0
alpha_max = 0
w_max = []
ll_max = -float("inf")
clf_best = 0

for i in range(8):
    w_tmp, ll_tmp, a_tmp, clf_tmp = reguralization2(X_train, X_test, y_train, 
                                   y_test, "l2", a)
    if ll_tmp > ll_max:
        ma = a_tmp
        alpha_max = a
        w_max = w_tmp
        ll_max = ll_tmp
        clf_best = clf_tmp
    a /= 10
    
print('テストデータの誤差が最小となるαは：', alpha_max)    
print('その時の正答率 = {:.3f}%'.format(ma))
print('その時の対数尤度 = {:.3f}'.format(ll_max))

/opt/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/base.py:340: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)


テストデータの誤差が最小となるαは： 0.0001
その時の正答率 = 66.859%
その時の対数尤度 = -17905.564


alphaがおよそ0.0001程度の大きさでテストデータの正答率が最大となることがわかった。最終テストを評価する。

In [24]:
# テストデータのラベルを予測
y_est = clf_best.predict(X_final_test)
y_est_proba = clf_best.predict_proba(X_final_test)
print_result(y_final_test, y_est, y_est_proba)
w_max

対数尤度 = -19905.787
正答率 = 67.032%
適合率（Precision） = 61.769%
再現率（Recall） = 46.114%
F1値（F1-score） = 52.806%


array([[-6.02241705e+00,  2.11673066e-01, -2.96004748e-01,
         1.23610447e-01,  7.36101981e-01, -4.32640110e-01,
         5.31959895e-01,  4.91666804e-03, -4.81557251e-01,
         4.65501216e-02, -3.96686183e-01, -2.04971364e-01,
        -2.35279080e-01,  3.69063092e-01, -4.04145776e-01,
        -9.89192510e-02, -2.02586191e-01,  7.01466603e-01,
        -1.33698461e-01,  1.57195241e-02,  1.97052211e-02,
         1.04207116e-01, -1.69535644e-01,  1.27291032e-01,
         1.57604101e-01, -1.44139391e-02, -1.69852330e-01,
        -6.66408514e-02, -3.72407389e-02,  3.14643853e-02,
         3.04249152e-02,  1.61849269e-01, -1.28816241e-01,
        -1.08262046e-01,  1.03391724e-01, -1.24021463e-02,
         1.97052211e-02,  4.68985500e-02,  7.35622963e-02,
        -1.60821535e-01,  4.01913689e-01, -2.96276882e-02,
         1.57604101e-01,  1.16486937e-02, -3.68563647e-01,
        -1.44139391e-02,  1.16970734e-01, -1.69852330e-01,
         1.86001408e-01, -1.00335195e-01, -5.83929091e-0

L1正則化項を加えたものとほぼ同程度の精度が得られた。正則化項なしのものよりこちらでも精度が向上している。正則化項なしの場合でも係数の大きなものはなかったためか、係数の値が小さくなる、というL2正則化の特徴はあまり見られていない。

## 3.  前処理について

#### Day 1にて、追加したダミー変数以外の予測に用いた変数については標準化を行っており、Day 2の課題では省略する。Day 1にてdelta, usd_goal_real, name_lengthにも相関が見られていないため、無相関化も不要であると判断した。

## 4. サポートベクターマシンによる予測モデル構築

In [25]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.model_selection import train_test_split

#### まずはC=10, kernel="rbf", gamma="scale"でモデルの構築を試みる。
#### 説明変数はロジスティック回帰モデルのL１正則化において重要であった5つを用いたもの、2-1のモデルで用いたものの二つを試す。
#### 計算量を減らすため、訓練データを基のデータの10%, テストデータを9%としている。

In [ ]:
#L1正則化で重要であったパラメータを用いる。
X3 = df_day2_3[['main_category_Music', 'name_length_std', 'delta_std', 'main_category_Technology', 'usd_goal_real_std']]

In [ ]:
#データの分割
#もとのデータの10%を訓練データとし、残りのデータの内の10%をテストデータとする。
X_train, X_res, y_train, y_res = train_test_split(X3, y, test_size = 0.9, random_state=1234)
X_test, X_res, y_test, y_res = train_test_split(X_res, y_res, test_size = 0.9, random_state=1234)

In [14]:
#SVMモデルの学習を行う。
C = 10
clf = SVC(C=C,kernel="rbf",gamma='scale')
clf.fit(X_train, y_train) 

SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [15]:
# 訓練データのラベルを予測し、精度を表示
y_est = clf.predict(X_train)
target_names = ['failed', 'successful']
print(metrics.classification_report(y_train, y_est,target_names=target_names))
conf_mat_train = pd.DataFrame(confusion_matrix(y_train, y_est), 
                        index=['正解 = failed', '正解 = successful'], 
                        columns=['予測 = failed', '予測 = successful'])

display(conf_mat_train)



              precision    recall  f1-score   support

      failed       0.63      0.89      0.74     19809
  successful       0.60      0.23      0.34     13355

    accuracy                           0.63     33164
   macro avg       0.61      0.56      0.54     33164
weighted avg       0.62      0.63      0.58     33164



,予測 = failed,予測 = successful
正解 = failed,17696,2113
正解 = successful,10233,3122


In [16]:
# テストデータのラベルを予測
y_est = clf.predict(X_test)
target_names = ['failed', 'successful']
print(metrics.classification_report(y_test, y_est,target_names=target_names))
conf_mat_test = pd.DataFrame(confusion_matrix(y_test, y_est), 
                        index=['正解 = failed', '正解 = successful'], 
                        columns=['予測 = failed', '予測 = successful'])

display(conf_mat_test)


              precision    recall  f1-score   support

      failed       0.63      0.89      0.74     17847
  successful       0.58      0.23      0.33     12001

    accuracy                           0.62     29848
   macro avg       0.61      0.56      0.53     29848
weighted avg       0.61      0.62      0.57     29848



,予測 = failed,予測 = successful
正解 = failed,15873,1974
正解 = successful,9276,2725


#### SVMのモデルを構築することができた。グリッドサーチにより、パラメータを最適化した後、最適なパラメータのモデルで再度学習を行い、ロジスティック回帰と比較する。
#### 次は説明変数2-1のものを利用する。

In [26]:
#データの分割、次は2-1のロジスティック回帰に用いた説明変数をすべて利用する。
X_train, X_res, y_train, y_res = train_test_split(X, y, test_size = 0.9, random_state=1234)
X_test, X_res, y_test, y_res = train_test_split(X_res, y_res, test_size = 0.9, random_state=1234)

In [30]:
#グリッドサーチによるパラメータの最適化
parameters = {'kernel':['rbf'], 'C':[0.1, 1, 10], 'gamma':[0.1, 1, 10]} 
model = SVC() 
clf = GridSearchCV(model, parameters, cv=3,)
clf.fit(X_train, y_train)
print(clf.best_params_, clf.best_score_)

{'C': 10, 'gamma': 0.1, 'kernel': 'rbf'} 0.6456700036183813


In [33]:
clf3 = SVC(**clf.best_params_)
clf3.fit(X_train, y_train)
print("識別精度=",clf3.score(X_test, y_test))

識別精度= 0.6394277481823969


In [37]:
import pickle 
with open('SVM model', 'wb') as web:
    pickle.dump(clf3, web)

In [38]:
import pickle
with open('SVM model', 'rb') as web:
    clf4 = pickle.load(web)

パラメータのチューニングを行うことができた。データ数を減らしたためか、汎化誤差が訓練誤差よりも大きくなっており、過学習している可能性があある。パラメータのチューニングも探索範囲がまだ十分ではない可能性もあり、今後さらに精度を上げられる可能性がある。

- ロジスティック回帰　正則化項なし　62%
- ロジスティック回帰　L１正則化     67%
- ロジスティック回帰　L２正則化     67%
- SVM                               64% 

今回行った学習の範囲ではL1, L2正則化項を加えたロジスティック回帰が最も汎化誤差が小さいという結果であった。